In [3]:
import os
import re

import pandas as pd

In [9]:
def get_text(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

In [22]:
df = pd.read_csv('items.csv', encoding='utf-8')
df.head()

,path,id,author,title,date
0,output/619049.txt,619049,ปาลญ์ ชญา,ดาวโจนส์บวกก่อนหยุดยาววันประธานาธบดีสหรัฐ,2018-02-17
1,output/617815.txt,617815,จรัญญา,กสทช. เสนอพักชำระค่าใบอนุญาติทีวีดิจิทัล 3 ปี,2018-02-13
2,output/617895.txt,617895,กอน-วิ-กา,ลดราคาขายปลีกน้ำมันทุกชนิด 50 สตางค์ เว้น E85 ...,2018-02-13
3,output/617950.txt,617950,MONO29 NEWS,แฟนๆ อดใจนิด! สวนสยามปิดปรับปรุงใหญ่ ก่อนเผยโฉ...,2018-02-13
4,output/616938.txt,616938,กอน-วิ-กา,ถกด่วน! หลังสื่อญี่ปุ่นตีข่าว แรงงานไทยติดสารเ...,2018-02-08


In [149]:
company_pattern = r'(?P<full>'+r'|'.join([
    r'บริษัท(?P<key_0>(?:.(?!บริษัท|ที่|หรือ|ใน))+?)จำกัด(?:\s*\(\s*มหาชน\s*\))?(?:\s*\(\s*.{2,20}\s*\))?',
    r'บริษัท(?P<key_1>[0-9\-\s]*[a-zA-Z][a-zA-Z0-9\-\(\)\.\,\s]*[a-zA-Z0-9\.\,])(?:\s*\(.{2,20}\))?',
])+r')'

In [150]:
entities = dict()
fullnames = dict()

for i, r in df.iterrows():
    text = get_text(r.path)
    for match in re.findall(company_pattern, get_text(df.path[i])):
        full, key1, key2 = map(str.strip, match)
        key = key1 or key2
        
        entities[key] = entities.get(key, list())
        entities[key].append(r.id)
        
        fullnames[key] = fullnames.get(key, set())
        fullnames[key].add(full)

In [152]:
lim = 250
for k in fullnames:
    s = '{}: {}'.format(k, ', '.join(fullnames[k]))
    print(s[:lim], ('...' if len(s) > lim else ''))
    print()

ปตท.: บริษัท ปตท.จำกัด   (มหาชน), บริษัทปตท. จำกัด (มหาชน), บริษัท ปตท. จำกัด(มหาชน), บริษัท ปตท. จำกัด, บริษัท ปตท. จำกัด  (มหาชน), บริษัท  ปตท. จำกัด (มหาชน), บริษัท ปตท. จำกัด (มหาชน), บริษัท ปตท.จำกัด (มหาขน), บริษัท  ปตท.  จำกัด (มหาชน), บริษัท  ...

บางจาก คอร์ปอเรชั่น: บริษัทบางจาก คอร์ปอเรชั่น จำกัด (มหาชน), บริษัทบางจาก คอร์ปอเรชั่น จำกัด(มหาชน), บริษัท บางจาก คอร์ปอเรชั่น จำกัด (มหาชน) 

สยามพาร์คซิตี้: บริษัทสยามพาร์คซิตี้ จำกัด 

เอกชัยการแพทย์: บริษัท เอกชัยการแพทย์ จำกัด (มหาชน) 

เอกชัย อินเตอร์เนชั่นแนล: บริษัท เอกชัย อินเตอร์เนชั่นแนล จำกัด 

หลักทรัพย์กสิกรไทย: บริษัทหลักทรัพย์กสิกรไทย จำกัด, บริษัทหลักทรัพย์กสิกรไทย จำกัด (มหาชน), บริษัทหลักทรัพย์กสิกรไทย  จำกัด 

อยุธยา แคปปิตอล เซอร์วิสเซส: บริษัท อยุธยา แคปปิตอล เซอร์วิสเซส จำกัด 

มารัตน์ทรานสปอร์ต: บริษัท มารัตน์ทรานสปอร์ต จำกัด 

โมโน บรอดคาซท์: บริษัท โมโน บรอดคาซท์ จำกัด, บริษัท โมโน บรอดคาซท์ จำกัด (Mono/Mthai), บริษัทโมโน บรอดคาซท์ จำกัด, บริษัท โมโน บรอดคาซท์ จำกัด (คนกลาง) 

หลักทรัพย์ กสิกรไทย: บริษัทหลั

โมโน บรอดคาชท์: บริษัท โมโน บรอดคาชท์  จำกัด 

ไทยเพรซิเดนท์ฟูดส์: บริษัท ไทยเพรซิเดนท์ฟูดส์ จำกัด (มหาชน) 

ร่วม และรับรู้รายได้งานใหม่ๆ บล.เมย์แบงก์ กิมเอ็ง (ประเทศไทย): บริษัทร่วม และรับรู้รายได้งานใหม่ๆ บล.เมย์แบงก์ กิมเอ็ง (ประเทศไทย) จำกัด (มหาชน) 

เป็ปซี่-โคล่า (ไทย) เทรดดิ้ง: บริษัท เป็ปซี่-โคล่า (ไทย) เทรดดิ้ง จำกัด 

หลักทรัพย์จัดการกองทุน ทิสโก้: บริษัทหลักทรัพย์จัดการกองทุน ทิสโก้ จำกัด 

Diner: บริษัท Diner, บริษัทDiner 

ยูโรเปี้ยนฟู้ด: บริษัท ยูโรเปี้ยนฟู้ด  จำกัด (มหาชน) 

เอเชียโกลเด้นไรท์: บริษัท เอเชียโกลเด้นไรท์ จำกัด 

วายแอลจี บูลเลี่ยน แอนด์ ฟิวเจอร์ส: บริษัท วายแอลจี บูลเลี่ยน แอนด์ ฟิวเจอร์ส จำกัด 

เออาร์ไอพี: บริษัท เออาร์ไอพี จำกัด (มหาชน) 

เจ.ไอ.บี. คอมพิวเตอร์ กรุ๊ป: บริษัท เจ.ไอ.บี. คอมพิวเตอร์ กรุ๊ป จำกัด 

โตชิบา ไทยแลนด์: บริษัท โตชิบา ไทยแลนด์ จำกัด 

หลักทรัพย์ ฟิลลิป (ประเทศไทย): บริษัทหลักทรัพย์ ฟิลลิป (ประเทศไทย) จำกัด (มหาชน) 

อาร์เอส เทเลวิชั่น: บริษัท อาร์เอส เทเลวิชั่น จำกัด 

RS: บริษัท RS 

เซ็นทรัลรีเทล คอร์ปอเรชั่น: บริษัทเซ็นทรัลรีเทล 

In [154]:
lim = 250
for k in entities:
    s = '{}: {}'.format(k, entities[k])
    print(s[:lim], ('...' if len(s) > lim else ''))
    print()

ปตท.: [617895, 618598, 616864, 615448, 611637, 609300, 607635, 605718, 606619, 603157, 600547, 597933, 597137, 593479, 593312, 592125, 590461, 590838, 589888, 589472, 587693, 587129, 584451, 585345, 583679, 580731, 574682, 574017, 574246, 572511, 571 ...

บางจาก คอร์ปอเรชั่น: [617895, 618598, 616864, 607184, 600547, 597933, 595474, 591786, 592125, 590838, 589888, 589472, 585345, 580731, 574017, 572511, 569757, 568888, 567550, 565657, 563694, 562053, 562467, 561008, 560168, 557674] 

สยามพาร์คซิตี้: [617950] 

เอกชัยการแพทย์: [617158] 

เอกชัย อินเตอร์เนชั่นแนล: [617158] 

หลักทรัพย์กสิกรไทย: [617443, 583089, 582425, 482225, 478857, 476630, 469645, 393102, 290986] 

อยุธยา แคปปิตอล เซอร์วิสเซส: [616924, 616924, 281565] 

มารัตน์ทรานสปอร์ต: [612743] 

โมโน บรอดคาซท์: [611845, 611845, 485152, 465134, 465134, 465134, 446624, 444486, 406379, 380884, 331793, 327724, 315862, 302541, 302541, 299583, 297540, 293941, 293216, 280735, 280735, 280390, 269918] 

หลักทรัพย์ กสิกรไทย: [610545, 540442,


ผ่าน The Central Depository (PTE) Limited มีจำนวน: [346175] 

กรุงเทพ วิทยุโทรทัศน์: [343190] 

SASA Polyester Sanayi A.S. (SASA: [343522, 342337] 

กรุงเทพวิทยุโทรทัศน์: [343730] 

บีอีซีเวิลด์: [343730] 

ทีวีไดเร็ค (มาเลเซีย): [343792] 

โนเบิ้ล: [335531] 

เอเซียติก อุตสาหกรรมเกษตร: [335846] 

เค.มาสเตอร์: [331950] 

อมตะ คอร์ปอเรชัน: [332400, 277723] 

คลาสสิก โกลด์ ฟิวเจอร์ส: [332385] 

ไทยเดินเรือทะเล: [330748] 

BECL: [331670] 

ซีคิว อะโกร: [329949] 

PwC: [329624] 

พรีเมี่ยม แมเนจเม้นท์: [329699] 

เดอะ ควอนท์ กรุ๊ป: [327996] 

วิจิตรภัณฑ์สวนปาล์ม: [327996] 

วีจี เอ็นเนอร์ยี: [327996] 

วิจิตภัณฑ์โลจิสติกส์: [327996] 

ไทย อะโกร เอ็นเนอร์ยี่: [328262] 

ลานนารีซอร์สเซส: [328262] 

พีเอ็ม เซ็นเตอร์: [326892, 326892] 

ซีเอฟจี เซอร์วิส: [326856, 325171, 325171, 304707, 258668] 

โมโน เจนเนอเรชั่น: [325605, 293941, 293216, 280735, 280735, 280390, 269918] 

เอช พลัส แชนแนล คอร์ปอเรชั่น: [325610, 325610, 325610] 

168 สตาร์: [325610, 291596] 

บลูสกาย แชนแนล: [325610, 325610] 
